In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# load the PCA model
sarcoma_pca_df = pd.read_csv('../Data/sarcoma-gene-exp-FPKM-zscore-no-label-pca.csv')
sarcoma_pca_df.shape

(206, 163)

In [2]:
# convert df to array
X = sarcoma_pca_df.to_numpy()
print(X)

[[ -7.1299698   19.54480635 -46.93123398 ...  -5.20201632  -6.68736361
    2.18412099]
 [ 21.38528472 -34.44671658 -29.18157729 ... -10.89246806   1.2741249
    7.78042611]
 [ 57.2761131    7.93957393   9.81154604 ...   3.43791233 -11.25802252
   -5.61020023]
 ...
 [ 78.94052539  27.36499822  12.99505297 ...   1.6504554   -0.35151221
    6.23520363]
 [  8.56846777 -21.26322469   9.74749633 ...  10.69648535   0.19657854
    3.12135705]
 [ 19.09385664 -14.72218889 -10.0079258  ...   4.28801274  -1.52572152
   -6.56352494]]


In [3]:
# read in labels
sarcoma_labels_df = pd.read_csv('../Data/sarcoma-gene-exp-FPKM-labels-only.csv')
sarcoma_labels_df.shape

(206, 1)

In [4]:
# Convert label df to np array
y_df = sarcoma_labels_df['label']
y = y_df.to_numpy()
print(y)

[4 2 2 1 5 3 2 4 4 3 4 4 4 3 0 3 1 4 2 4 4 0 2 4 3 3 3 2 0 4 4 5 3 3 2 4 4
 5 4 3 4 4 5 4 2 4 4 4 1 4 1 4 4 2 4 3 3 2 2 4 4 4 4 4 4 4 2 2 3 4 2 3 4 4
 3 4 3 2 4 3 2 3 4 4 3 4 3 4 3 4 4 1 3 4 4 4 0 4 3 4 3 3 3 5 2 0 3 3 1 1 4
 2 3 0 3 4 2 4 2 0 3 4 4 3 2 2 1 3 4 4 4 4 4 3 2 4 2 4 1 0 2 2 3 4 4 2 3 4
 4 3 1 3 4 3 4 2 2 1 3 2 2 0 4 2 1 4 2 1 4 3 3 4 2 4 2 4 3 4 2 1 3 4 2 4 4
 2 2 3 4 4 4 3 2 4 1 1 2 3 0 3 3 2 2 3 1 2]


In [5]:
# Get a count of the unique values in each categories to make sure there are enough to support cross-validation
unique_elements, counts_elements = np.unique(y, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[ 0  1  2  3  4  5]
 [10 17 44 50 80  5]]


In [6]:
# Use Stratified Kfold split
from sklearn.model_selection import StratifiedKFold,train_test_split
k_fold = StratifiedKFold(n_splits=4,shuffle=True,random_state=2019)
# Split training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2018, stratify=y)
print(len(X_train))
# Create k folds with training data
for train_index, cv_index in k_fold.split(np.zeros(len(X_train)), y_train.ravel()):
    X_train_fold, X_cv_fold = X_train[train_index,:], X_train[cv_index,:]
    y_train_fold, y_cv_fold = y_train[train_index], y_train[cv_index]

154


In [7]:
# KNN Classifier
from sklearn.neighbors import KNeighborsClassifier

# instantiate the model
neigh = KNeighborsClassifier(n_neighbors=7)
for train_index, cv_index in k_fold.split(np.zeros(len(X_train)), y_train.ravel()):
    X_train_fold, X_cv_fold = X_train[train_index,:], X_train[cv_index,:]
    y_train_fold, y_cv_fold = y_train[train_index], y_train[cv_index]
    neigh.fit(X_train_fold, y_train_fold) 
    # model evaluation for training set
    y_train_pred = neigh.predict(X_train_fold)
    y_test_pred = neigh.predict(X_cv_fold)
    # record training set accuracy
    print(neigh.score(X_train_fold, y_train_pred))
    # record generalization accuracy
    print(neigh.score(X_cv_fold, y_cv_fold))
print(neigh.score(X_test, y_test))


1.0
0.6341463414634146
1.0
0.6578947368421053
1.0
0.7894736842105263
1.0
0.7837837837837838
0.6346153846153846


In [8]:
# SVM Classifier
from sklearn.svm import SVC

svc = SVC(kernel='rbf', gamma='scale', C=1.0)
for train_index, cv_index in k_fold.split(np.zeros(len(X_train)), y_train.ravel()):
    X_train_fold, X_cv_fold = X_train[train_index,:], X_train[cv_index,:]
    y_train_fold, y_cv_fold = y_train[train_index], y_train[cv_index]
    svc.fit(X_train, y_train) 
    # model evaluation for training set
    y_train_pred = svc.predict(X_train_fold)
    y_test_pred = svc.predict(X_cv_fold)
    # record training set accuracy
    print(svc.score(X_train_fold, y_train_fold))
    # record generalization accuracy
    print(svc.score(X_cv_fold, y_cv_fold))
print(svc.score(X_test, y_test))


0.9469026548672567
0.9024390243902439
0.9396551724137931
0.9210526315789473
0.9137931034482759
1.0
0.9401709401709402
0.918918918918919
0.7115384615384616


In [9]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(max_depth=4, random_state=0, n_estimators=100) 
for train_index, cv_index in k_fold.split(np.zeros(len(X_train)), y_train.ravel()):
    X_train_fold, X_cv_fold = X_train[train_index,:], X_train[cv_index,:]
    y_train_fold, y_cv_fold = y_train[train_index], y_train[cv_index]
    rfc.fit(X_train_fold, y_train_fold) 
    # model evaluation for training set
    y_train_pred = rfc.predict(X_train)
    y_test_pred = rfc.predict(X_cv_fold)
    # record training set accuracy
    print(rfc.score(X_train_fold, y_train_fold))
    # record generalization accuracy
    print(rfc.score(X_cv_fold, y_cv_fold))
print(rfc.score(X_test, y_test))

0.9911504424778761
0.4878048780487805
0.9741379310344828
0.6052631578947368
0.9568965517241379
0.5526315789473685
0.9572649572649573
0.5675675675675675
0.46153846153846156


In [10]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression

lrc = LogisticRegression(multi_class='multinomial', solver='sag', max_iter=1000, C=1.0, random_state=0, penalty='l2') 
for train_index, cv_index in k_fold.split(np.zeros(len(X_train)), y_train.ravel()):
    X_train_fold, X_cv_fold = X_train[train_index,:], X_train[cv_index,:]
    y_train_fold, y_cv_fold = y_train[train_index], y_train[cv_index]
    lrc.fit(X_train_fold, y_train_fold) 
    # model evaluation for training set
    y_train_pred = lrc.predict(X_train)
    y_test_pred = lrc.predict(X_cv_fold)
    # record training set accuracy
    print(lrc.score(X_train_fold, y_train_fold))
    # record generalization accuracy
    print(lrc.score(X_cv_fold, y_cv_fold))
print(lrc.score(X_test, y_test))

1.0
0.7317073170731707
1.0
0.631578947368421
1.0
0.868421052631579
1.0
0.8108108108108109
0.7692307692307693


In [11]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [12]:
# Grid search for KNN
from sklearn.neighbors import KNeighborsClassifier

# Specify number of neighbors as hyperparameter
neighbors = range(1, 12)

# Create hyperparameter options
hyperparameters = dict(n_neighbors=neighbors)
gs_knn = GridSearchCV(KNeighborsClassifier(), hyperparameters, cv=4, scoring="accuracy", verbose=0)
# Fit search
best_model_knn = gs_knn.fit(X_train, y_train)
    
# View best hyperparameters
print('Best Number of Neighbors:', best_model_knn.best_estimator_.get_params()['n_neighbors'])

# Test the model and report Accuracy score
print('Accuracy Score:', accuracy_score(y_test, best_model_knn.predict(X_test)))


Best Number of Neighbors: 5
Accuracy Score: 0.5961538461538461


/Users/jeffreydelong/plaidml-venv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [13]:
# Grid search for SVM
from sklearn.svm import SVC

# Create hyperparameter space
C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
# Create hyperparameter options
hyperparameters = dict(C=C_range, gamma=gamma_range)
gs_svm = GridSearchCV(SVC(kernel='rbf'), hyperparameters, cv=4, scoring="accuracy", verbose=0)
# Fit search
best_model_svm = gs_svm.fit(X_train, y_train)

# View best hyperparameters
print('Best value of C', best_model_svm.best_estimator_.get_params()['C'])
print('Best value of gamma', best_model_svm.best_estimator_.get_params()['gamma'])

# Test the model and report Accuracy score
print('Accuracy Score:', accuracy_score(y_test, best_model_svm.predict(X_test)))

Best value of C 100.0
Best value of gamma 1e-05
Accuracy Score: 0.7884615384615384


/Users/jeffreydelong/plaidml-venv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [14]:
# Grid search for RF
from sklearn.ensemble import RandomForestClassifier

# Create hyperparameter space
maxdepth=range(1, 12)
# Create hyperparameter options
hyperparameters = dict(max_depth=maxdepth)
gs_rfc = GridSearchCV(RandomForestClassifier(random_state=0, n_estimators=100), hyperparameters, cv=4, scoring="accuracy", verbose=0)
# Fit search
best_model_rfc = gs_rfc.fit(X_train, y_train)
    
# View best hyperparameters
print('Best value of Max depth', best_model_rfc.best_estimator_.get_params()['max_depth'])

# Test the model and report Accuracy score
print('Accuracy Score:', accuracy_score(y_test, best_model_rfc.predict(X_test)))

Best value of Max depth 5
Accuracy Score: 0.5961538461538461


/Users/jeffreydelong/plaidml-venv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [15]:
# Grid search for LR
from sklearn.linear_model import LogisticRegression

# Create hyperparameter space
C = np.logspace(0, 4, 10)
# Create hyperparameter options
hyperparameters = dict(C=C)
gs_lrc = GridSearchCV(LogisticRegression(multi_class='multinomial', solver='sag', max_iter=5000, random_state=0, penalty='l2') , hyperparameters, cv=4, scoring="accuracy", verbose=0)
# Fit search
best_model_lrc = gs_lrc.fit(X_train, y_train)
    
# View best hyperparameters
print('Best value of C', best_model_lrc.best_estimator_.get_params()['C'])

# Test the model and report Accuracy score
print('Accuracy Score:', accuracy_score(y_test, best_model_lrc.predict(X_test)))

/Users/jeffreydelong/plaidml-venv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best value of C 1.0
Accuracy Score: 0.8269230769230769


In [16]:
# Best model is LR with C = 1
lrc = LogisticRegression(multi_class='multinomial', solver='sag', max_iter=1000, C=1.0, random_state=0, penalty='l2') 
lrc.fit(X_train, y_train) 

y_pred = lrc.predict(X_test)
from sklearn.metrics import classification_report
class_names = ["Class {}".format(i) for i in range(6)]
print(classification_report(y_test, y_pred, target_names=class_names))


              precision    recall  f1-score   support

     Class 0       1.00      1.00      1.00         3
     Class 1       0.50      0.25      0.33         4
     Class 2       0.60      0.82      0.69        11
     Class 3       0.92      0.92      0.92        13
     Class 4       0.94      0.85      0.89        20
     Class 5       1.00      1.00      1.00         1

    accuracy                           0.83        52
   macro avg       0.83      0.81      0.81        52
weighted avg       0.84      0.83      0.82        52



In [17]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[ 3,  0,  0,  0,  0,  0],
       [ 0,  1,  3,  0,  0,  0],
       [ 0,  0,  9,  1,  1,  0],
       [ 0,  1,  0, 12,  0,  0],
       [ 0,  0,  3,  0, 17,  0],
       [ 0,  0,  0,  0,  0,  1]])

In [18]:
# Voting classifier
def votingClassifiers():
    # This example is taken from https://github.com/ageron/handson-ml/blob/master/07_ensemble_learning_and_random_forests.ipynb 


    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import VotingClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.svm import SVC
    from sklearn.metrics import accuracy_score
    
    log_clf = LogisticRegression(multi_class='multinomial', solver='sag', max_iter=1000, C=1.0, random_state=0, penalty='l2') 

    svm_clf = SVC(kernel='rbf', gamma=1e-05, C=100)
  
    svm2_clf = SVC(kernel='rbf', gamma='scale', C=1)
  
    voting_clf = VotingClassifier(
            estimators=[('lr', log_clf), ('svc', svm_clf), ('svc2', svm2_clf)],
            voting='hard')
     
    for clf in (log_clf, svm_clf, svm2_clf, voting_clf):
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

In [19]:
votingClassifiers()

LogisticRegression 0.8269230769230769
SVC 0.7884615384615384
SVC 0.7115384615384616
VotingClassifier 0.7884615384615384


In [20]:
def baggingClassifiers():
    # This example is taken from https://github.com/ageron/handson-ml/blob/master/07_ensemble_learning_and_random_forests.ipynb 


    from sklearn.ensemble import BaggingClassifier
    from sklearn.metrics import accuracy_score
    from sklearn.linear_model import LogisticRegression

    bag_clf = BaggingClassifier(
        LogisticRegression(multi_class='multinomial', solver='sag', max_iter=2000, C=1.0, penalty='l2'), n_estimators=100,
            max_samples=100, bootstrap=True, n_jobs=-1, random_state=42)
    bag_clf.fit(X_train, y_train)
    y_pred = bag_clf.predict(X_test)
    print('Bagging LR',accuracy_score(y_test, y_pred))

In [21]:
baggingClassifiers()

Bagging LR 0.8269230769230769


In [22]:
def adaBoostClassifiers():
    # This example is taken from https://github.com/ageron/handson-ml/blob/master/07_ensemble_learning_and_random_forests.ipynb 

    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.metrics import accuracy_score
    from sklearn.linear_model import LogisticRegression

    ada_clf = AdaBoostClassifier(
        LogisticRegression(multi_class='multinomial', solver='sag', max_iter=3000, C=1.0, penalty='l2'), n_estimators=100,
             random_state=42)
    ada_clf.fit(X_train, y_train)
    y_pred = ada_clf.predict(X_test)
    print('AdaBoost LR', accuracy_score(y_test, y_pred))

In [23]:
adaBoostClassifiers()

AdaBoost LR 0.8076923076923077
